In [2]:
# Vector Data impimentation
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [3]:
loader = PyPDFLoader("../test_documents/attention.pdf")

In [6]:
document = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents =  text_splitter.split_documents(document)
vectors = FAISS.from_documents(documents, embeddings)

In [8]:
retriver = vectors.as_retriever()

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "Database_Search",
                      """Search for information about the research paper. You should use this tool to get the information about the relevant research 
                      paper.""")